In [1]:
#Irei importar todas as bibliotecas necessárias para o desenvolvimento do projeto
import pandas as pd
import numpy as np
import random
from  more_itertools import unique_everseen
from scipy.optimize import fmin_cg

In [2]:
#Nesta seção irei definir as funções necessárias para fazer a fatorização da matriz inicial além 
#da função para calcular a qualidade das previsões.

def normalize_ratings(ratings):
    """
    Essa função tem como objetivo principal evitar o overfitting dos dados 
    Dado um array qualquer: essa função irá subtrair de cada elemento o valor médio do array dado.
   
    :Imput: ratings = 2D array com as notas dadas
    :OutPut: array de entrada  normalizado 
             e o valor médio das notas
    """
    mean_ratings = np.nanmean(ratings, axis=0)
    return ratings - mean_ratings, mean_ratings


def cost(X, *args):
    """
    Essa função deve ser minimizada (durante as diversas interações) para que seja obtido o melhor resultado final 
    (que são as mtrizes que descrevem os usuários e os filmes)
    Imput: X = matriz que deve ser fatorada, num_users (quantidade de usuários), num_products(quantidade de filmes), 
    num_features (quantidade de gêneros), ratings (matriz com as notas), mask (define onde estão os elementos
    vazios), regularization_amount
    Output: valor da função custo, bem como as matrizes P e Q que darão origem às matrizes com as caracteristicas 
    do filmes e usuários
    """
    num_users, num_products, num_features, ratings, mask, regularization_amount = args

    # Separar o array X em dois
    P = X[0:(num_users * num_features)].reshape(num_users, num_features)
    Q = X[(num_users * num_features):].reshape(num_products, num_features)
    Q = Q.T

    # Calculo da função custo
    return (np.sum(np.square(mask * (np.dot(P, Q) - ratings))) / 2) + ((regularization_amount / 2.0) * np.sum(np.square(Q.T))) + ((regularization_amount / 2.0) * np.sum(np.square(P)))


def gradient(X, *args):
    """
    Define o método para minimizar a função cost (cost)
    
    Imput: X = matriz que deve ser fatorada, num_users (quantidade de usuários), num_products(quantidade de filmes), 
    num_features (quantidade de gêneros), ratings (matriz com as notas), mask (define onde estão os elementos
    vazios), regularization_amount
    Output: o gradiente com a matriz X atualizada
   
    """
    num_users, num_products, num_features, ratings, mask, regularization_amount = args

    # Separar a matriz X em duas
    P = X[0:(num_users * num_features)].reshape(num_users, num_features)
    Q = X[(num_users * num_features):].reshape(num_products, num_features)
    Q = Q.T

    # Calcular o gradiante para as matrizes P e Q
    P_grad = np.dot((mask * (np.dot(P, Q) - ratings)), Q.T) + (regularization_amount * P)
    Q_grad = np.dot((mask * (np.dot(P, Q) - ratings)).T, P) + (regularization_amount * Q.T)

    # Output (o gradiente com a matriz X atualizada)
    return np.append(P_grad.ravel(), Q_grad.ravel())


def low_rank_matrix_factorization(ratings, mask=None, num_features=18, regularization_amount=0.5):
    """
    A partir da matriz original irei tentar determinar as caracteristicas dos filmes e preferencias dos usuários
    em 18 categogias. O resultado final será uma matriz [2000, 18] que descreve as preferencias do usuários e uma 
    segunda matriz [18, 150] de descreve as caracteristicas dos filmes em 18 generos
    """
    num_users, num_products = ratings.shape

    # Se nenhuma mascara for dada, os lugares vagos serão usados como mascara
    if mask is None:
        mask = np.invert(np.isnan(ratings))

    # substituir lugares vagos na matriz (NaN) por zero
    ratings = np.nan_to_num(ratings)

    # Criar as P e Q com numeros aleatórios, entre 0 e 1. Eu multiplo por 0.2 pois depois de alguns testes percebi
    #que ao criarmos matrizes com valores menores iremos obter uma melhor convergência
    np.random.seed(0)
    P = 0.2*np.random.randn(num_users, num_features)
    Q = 0.2*np.random.randn(num_products, num_features)

    # juntar os arrays criados P e Q, pois irei utilizar uma função fmin_cg (e essa função exige apenas uma array)
    initial = np.append(P.ravel(), Q.ravel())

    # Criar uma lista com os parametros necessários para chamar as funções
    args = (num_users, num_products, num_features, ratings, mask, regularization_amount)
    
    #usar a função fmin_cg para minimizar a função custo (cost), usando a função gradient definida acima 
    #A quantidade máxima de interações para chegarmos ao critério de convergência (definido pela variável
    #regularization_amount) é estabelecido pela variável maxiter (neste caso 200000 interações)    
    X = fmin_cg(cost, initial, fprime=gradient, args=args, maxiter=50000)

    #Irei dividir a matriz X para obter as matrizes final
    #nP=descreve as preferencias dos usuário
    #nQ=descreve as caracteristicas de cada filme em cada genero 
    nP = X[0:(num_users * num_features)].reshape(num_users, num_features)
    nQ = X[(num_users * num_features):].reshape(num_products, num_features)
    
    return nP, nQ.T

#Função para calcular o quão próximo a matriz com as notas calculadas é da matriz com as notas originais 
def RMSE(real, predicted):

    return np.sqrt(np.nanmean(np.square(real - predicted)))


In [3]:
#Nesta seção irei ler os dados do arquivo ratings.csv e selecionar os usuários com o maior número 
#possível de avaliações e em seguida os filmes mas avaliados. Isso é necessário pois não consegui 
#processar todos os dados, desta maneira estou escolendo os filmes e usuários que me permita ter 
#uma matriz inicial com o menor quantidade possível de espaços vazios. Assim o problema terá uma
#melhor convergência 

#ler o arquivo com os dados inicial
data_table1 = pd.read_csv("ratings.csv", usecols = ["userId","movieId", "rating"] )

#irei contar quantas avaliações foram dadas por cada usuário
user_rating_amount=pd.DataFrame((data_table1.groupby('userId').userId.count()).values)

#irei criar um data frame apenas com as identidades dos usuários
user=list(unique_everseen(data_table1.iloc[:, 0].values))
user=pd.DataFrame({'userId': user})

#adicionar uma segunda coluna no dataframe user com a quantidade de avaliações dada por cada usuário
user['user_rating_amount'] = user_rating_amount

#selecionar os usuário com um número minímo de avaliações
selected_user = (user.loc[user['user_rating_amount'] > 900])

#irei ler os dados iniciais novamente, porém irei selecionar os usuários que estão no dataframe 
#selected_user
user=list(selected_user.iloc[:,0])
sorted_final=data_table1.loc[data_table1['userId'].isin(user)]

#irei repetir o procedimento inicial (feito para selecionar os usuário que deram mais avaliações), 
#mas agora irei selecionar os filmes que receberam mais avaliações  
film_rating_amount=pd.DataFrame((sorted_final.groupby('movieId').movieId.count()).values)

film=sorted_final.iloc[:, 1]

film=pd.DataFrame({'movieId': list(unique_everseen(film.values))})
film['film_rating_amount'] = film_rating_amount

selected_film = (film.loc[film['film_rating_amount'] > 500])
movie=list(selected_film.iloc[:,0])

sorted_final=sorted_final.loc[sorted_final['movieId'].isin(movie)]

###########
##irei organizar os dados em uma matriz, se algum usuário avaliou o mesmo filme mais de uma vez
#irei salvar apenas a média dos valores
rating = pd.pivot_table(sorted_final, index='userId', columns='movieId', aggfunc=np.average)

#irei salavar os dados salvos
rating.iloc[:, :].to_csv("rating_final.csv")
rating.iloc[:, :].to_csv("rating_final_1.csv", header=None)

In [4]:
#ler o arquivo salvo na etapa anterior
rating = pd.read_csv("rating_final_1.csv", header=None)
ratings=rating.values[:, 1:]

#fazer a fatorização da matriz ratings
USER, FILM = low_rank_matrix_factorization (ratings)

#Calcular a matriz com as notas previstas pelo modelo
predicted_ratings = np.matmul(USER, FILM)
predicted_ratings = np.round(predicted_ratings, 2)
predicted_ratings[predicted_ratings > 5.0] = 5.0
predicted_ratings[predicted_ratings < 0.1] = 0.0

#Calcular a proximidade entre as notas dadas e previstas
rmse_training = RMSE(ratings,predicted_ratings)
print("Training RMSE: {}".format(rmse_training))

#Salvar as tres matrizes encontradas nesta etapa do programa
np.savetxt("predicted_1.csv", predicted_ratings, delimiter=",")
np.savetxt("user_feature.csv", USER, delimiter=",")
np.savetxt("film_feature.csv", FILM, delimiter=",")

         Current function value: 190994.792012
         Iterations: 7017
         Function evaluations: 10700
         Gradient evaluations: 10689
Training RMSE: 0.6549772508032228


In [5]:
#irei conferir a qualidade do modelo, para isso irei dividir (de maneira aleatória) a matriz inicial
#(ratings) em duas matrizes, uma para o treino do modelo e a segunda matriz para testar a qualidade
#das notas previtas

#divisão aleatória da matriz inical
ratings=pd.DataFrame(ratings)

notas=ratings.values[:,:]
row=np.shape(notas)[0]
col=np.shape(notas)[1]

test=np.zeros((row, col))
train=np.zeros((row, col))
            
for i in range (0, row):
    for j in range (0, col):
        if 5.0>=(notas[i,j])>=0.0:
            a=random.uniform(0, 1)
            if a<0.7:
                test[i,j]=np.nan
                train[i,j]=notas[i,j]
            if a>=0.7:
                test[i,j]=notas[i,j]
                train [i,j] = np.nan
        if not 5.0>=(notas[i,j])>=0.0:
            test[i,j]=np.nan
            train[i,j]=np.nan 

#treinar o modelo com a matriz traino (train)
USER, FILM = low_rank_matrix_factorization (train)

#prever as notas
predicted_ratings_train = np.matmul(USER, FILM)
predicted_ratings_train = np.round(predicted_ratings_train, 2)
predicted_ratings_train[predicted_ratings_train > 5.0] = 5.0
predicted_ratings_train[predicted_ratings_train < 0.1] = 0.0

#comparar as notas calculadas com as notas da matriz treino e teste
rmse_training = RMSE(train,predicted_ratings_train)
rmse_test = RMSE(test,predicted_ratings_train)
print("Training RMSE: {}".format(rmse_training))
print("Test RMSE: {}".format(rmse_test))

         Current function value: 127776.814152
         Iterations: 9209
         Function evaluations: 13870
         Gradient evaluations: 13869
Training RMSE: 0.6348174177266133
Test RMSE: 0.7835624205985096


In [12]:
#inicio do sistema de indicações. Esse primeiro modelo irá se basear no gosto do usuário para indicar
#um novo filme. Ou seja, as indicações serão baseadas em indicações dadas pelos usuários

#Irei ler os arquivos necessários 
already=pd.read_csv('ratings.csv', usecols = ["userId","movieId", "rating"])
rating = pd.read_csv("rating_final.csv", header=None )
movies = pd.read_csv("movies.csv", usecols = ["movieId","title", "genres"])

#caso queria ver o tamanho da matriz rating
print (np.shape(rating))

#lista dos usuários selecionados
userID = rating.values[3:, 0]
userID = list(map(int, userID))

#lista dos filmes selecionados
movieIDS=rating.values[1, 1:]
movieIDS = list(map(int, movieIDS))

#ler as informações dos filmes selecionados
movie = pd.DataFrame(movies.loc[movies['movieId'].isin(movieIDS)])
movies = pd.read_csv("movies.csv",index_col='movieId')

predicted_ratings = (pd.read_csv("predicted_1.csv", header = None )).values

#caso queira saber quais usuários foram selecionados 
print (userID[200:230])
#o programa vai perguntar pelo usuário para receber a indicação
#print("Entre com o número do usuário:")
user_id_to_search = int(input())

#Imprimir alguns filmes que foram avaliados pelo usuário selecionado 
user_index = userID.index(user_id_to_search)
reviewed_movies = already[already['userId'] == user_id_to_search]
reviewed_movies = reviewed_movies.join(movies, on='movieId')
reviewed_movies = reviewed_movies.sort_values(by=['rating'], ascending=False)
print (reviewed_movies.head(15))

#Excluindo os filmes que o usário já avaliou, irei imprimir os filmes que podem agradar o usuário
#considerando a nota prevista
user_ratings = predicted_ratings[user_index]
movie['rating'] = user_ratings
already_reviewed = reviewed_movies['movieId']
recommended = movie[movie.index.isin(already_reviewed) == False]
recommended = recommended.sort_values(by=['rating'], ascending=False)
print ("As recomendações são:")
print (recommended.head(5))

(2343, 1943)
[11333, 11348, 11434, 11517, 11538, 11560, 11719, 11748, 11794, 11886, 11900, 11902, 12026, 12044, 12080, 12131, 12140, 12158, 12239, 12490, 12618, 12644, 12653, 12767, 12793, 12802, 12950, 12973, 13064, 13174]


 12140


         userId  movieId  rating                             title  \
1798397   12140   112852     5.0    Guardians of the Galaxy (2014)   
1798240   12140    36535     5.0  Everything Is Illuminated (2005)   
1797530   12140     1274     5.0                      Akira (1988)   
1798309   12140    56367     5.0                       Juno (2007)   
1797533   12140     1281     5.0        Great Dictator, The (1940)   
1798306   12140    55442     5.0                 Persepolis (2007)   
1798305   12140    55280     5.0     Lars and the Real Girl (2007)   
1798304   12140    55276     5.0            Michael Clayton (2007)   
1798301   12140    54995     5.0              Planet Terror (2007)   
1798300   12140    54503     5.0                   Superbad (2007)   
1798293   12140    53894     5.0                      Sicko (2007)   
1797562   12140     1449     5.0        Waiting for Guffman (1996)   
1798292   12140    53519     5.0                Death Proof (2007)   
1798286   12140    5

###### 

In [14]:
#Nessa etapa a recomendação será baseada no gênero do filme selecionado. Irei indicar com 
#caracteristicas próximoas às do filme selecionado (sistema recomendado para usuários novos)

#Irei ler os dados necessário e transpor a matriz com as caracteristicas dos filmes apenas para 
#simplicar a manipulação dos dados
film_feature = pd.read_csv("film_feature.csv", header=None)
film_feature=np.transpose(film_feature.values)
movies = pd.read_csv("movies.csv", usecols = ["movieId","title", "genres"])
rating = pd.read_csv("rating_final.csv", header=None )

#lista dos filmes filmes salvos
movieIDS=rating.values[1, 1:]
movieIDS = list(map(int, movieIDS))

#ler apenas as caracteristicas dos filmes selecionados
movie = pd.DataFrame(movies.loc[movies['movieId'].isin(movieIDS)])

#Informar a identidade de um filme
print (movieIDS[400:430])
#print("Entre com o filmeId para ter sua recomendação:")
movieId = int(input())

#Irei selecionar as caracteristicas do filme selecionado, calcular a diferença entre o filme 
#selecionado com os demais, e irei mostrar os filmes com as menore diferenças
movie_index = movieIDS.index(movieId)

current_movie_features = film_feature[movie_index]

difference = film_feature - current_movie_features

absolute_difference = np.abs(difference)

total_difference = np.sum(absolute_difference, axis=1)

movie['difference_score'] = total_difference

sorted_movie_list = movie.sort_values(by=['difference_score'], ascending=True)

print("Os 5 filmes sugeridos são (o primeiro filme da lista será o filme selecionado):")
print (sorted_movie_list.head(6))

[1342, 1344, 1345, 1348, 1350, 1356, 1359, 1360, 1361, 1367, 1370, 1371, 1372, 1373, 1374, 1375, 1377, 1380, 1385, 1386, 1389, 1391, 1393, 1405, 1407, 1422, 1427, 1431, 1432, 1436]


 1


Os 5 filmes sugeridos são (o primeiro filme da lista será o filme selecionado):
      movieId                                              title  \
0           1                                   Toy Story (1995)   
3027     3114                                 Toy Story 2 (1999)   
360       364                              Lion King, The (1994)   
2901     2987                    Who Framed Roger Rabbit? (1988)   
1075     1097                  E.T. the Extra-Terrestrial (1982)   
2001     2085  101 Dalmatians (One Hundred and One Dalmatians...   

                                                 genres  difference_score  
0           Adventure|Animation|Children|Comedy|Fantasy          0.000000  
3027        Adventure|Animation|Children|Comedy|Fantasy          1.172199  
360     Adventure|Animation|Children|Drama|Musical|IMAX          3.438745  
2901  Adventure|Animation|Children|Comedy|Crime|Fant...          3.490143  
1075                              Children|Drama|Sci-Fi        

In [15]:
# este sistema de recomendação irá mostrar os filmes que outros usuários (que gostaram do filme 
#selecionado) também gostaram

#ler os arquivos necessários
predicted_ratings = (pd.read_csv("predicted_1.csv", header = None )).values

movies = pd.read_csv("movies.csv", usecols = ["movieId","title", "genres"])
movieIDS=rating.values[1, 1:]
movieIDS = list(map(int, movieIDS))

#ler os dados dos filmes selecionados
movie = (movies.loc[movies['movieId'].isin(movieIDS)])
movie=pd.DataFrame(movie)

#mostrar alguns filmes selecionados 
print (movieIDS[1000:1010])

#selecionar um filme
print("Entre com o filmeId para ter sua recomendação:")
movieid = int(input())
column = movieIDS.index(movieid)

#Irei selecionar usuários que deram notas maiores ou iguais a 4 para o filme selecionado
accepted = predicted_ratings[predicted_ratings[:, column]>=4.0]

#irei calcular a média das notas recebidas por cada filmes, irei ordenar os filmes em ordem 
#decrescente, considerando a nota média, por ultimo irei mostrar o filmes que os demais usuários
#(que gostaram do filme selecionado) mais gostaram 
ave = np.average(accepted, axis=0)
ave=pd.DataFrame(ave)

movie_selected= movie['movieId']

movie['Rating_Mean'] = ave

#Mostrar o filme selecionado
movie_information = movie.iloc[column]
print("Filme procurado:")
print("Filme: {}".format(movie_information.title))
print("Genero: {}".format(movie_information.genres))

#mostrar as indicações
sorted_movie_list = movie.loc[movie['movieId'].isin([movieid])==False]
sorted_movie_list = sorted_movie_list.sort_values(by=['Rating_Mean'], ascending=False)
print("Pessoas que gostaram do filme selecionado também gostaram dos seguintes filmes:")
sorted_movie_list.head(10)

[4037, 4038, 4040, 4052, 4053, 4054, 4055, 4058, 4074, 4091]
Entre com o filmeId para ter sua recomendação:


 4040


Filme procurado:
Filme: Don't Tell Mom the Babysitter's Dead (1991)
Genero: Comedy
Pessoas que gostaram do filme selecionado também gostaram dos seguintes filmes:


,movieId,title,genres,Rating_Mean
1445,1485,Liar Liar (1997),Comedy,4.706250
1547,1599,Steel (1997),Action,4.530833
1861,1945,On the Waterfront (1954),Crime|Drama,4.462917
332,336,"Walking Dead, The (1995)",Drama|War,4.455417
1729,1805,Wild Things (1998),Crime|Drama|Mystery|Thriller,4.431667
1103,1126,Drop Dead Fred (1991),Comedy|Fantasy,4.404583
1018,1037,"Lawnmower Man, The (1992)",Action|Horror|Sci-Fi|Thriller,4.359583
747,760,Stalingrad (1993),Drama|War,4.354167
1291,1320,Alien³ (a.k.a. Alien 3) (1992),Action|Horror|Sci-Fi|Thriller,4.332083
21,22,Copycat (1995),Crime|Drama|Horror|Mystery|Thriller,4.325000
